<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this notebook to fine-tune the model with linear evaluation.

# Setup

## Basic Imports

In [ ]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

import gdown
import shutil

from tqdm import tqdm

import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

## Download Datasets

In [ ]:
checkpoints = {
    # MorphCLRSingle checkpoints
    "canny_single_pretrained": "stl10_canny_single_pretrained_resnet18_0050",
    "canny_single_random": "stl10_canny_single_random_resnet18_0050",
    "dexined_single_pretrained": "stl10_dexined_single_pretrained_resnet18_0050",
    "dexined_single_random": "stl10_dexined_single_random_resnet18_0050",
    "baseline_pretrained": "stl10_pretrained_resnet18_0050",
    "baseline_random": "stl10_random_resnet18_0050",
    "baseline_canny_random": "stl10_canny_single_random_resnet18_0050",
    "baseline_dexined_random": "stl10_dexined_single_random_resnet18_0050",
    # MorphCLRDual checkpoints
    "canny_dual_pretrained": "stl10_canny_dual_pretrained_resnet18_0050",
    "canny_dual_random": "stl10_canny_dual_random_resnet18_0050",
    "dexined_dual_pretrained": "stl10_dexined_dual_pretrained_resnet18_0050",
    "dexined_dual_random": "stl10_dexined_dual_random_resnet18_0050",
}

# Train the Classification Layer and Save the Checkpoint

In [ ]:
import itertools
from matplotlib import cm
from sklearn.manifold import TSNE

import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from Edge_images.generate_datasets import (
    STL10,
    CannyDataset,
    DexiNedTrainDataset,
    DexiNedTestDataset,
    DualDataset,
)
from models.morphclr import MorphCLRBase, MorphCLRSingleEval, MorphCLRDualEval
from sklearn.manifold import TSNE

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: ", device)

In [ ]:
data_root = "../datasets"
checkpoint_root = "../runs"
print("Data root: ", data_root)
print("Checkpoint root: ", checkpoint_root)

In [ ]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
    train_dataset = datasets.STL10(
        "../datasets", split="train", download=download, transform=transforms.ToTensor()
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=False,
        shuffle=shuffle,
    )

    test_dataset = datasets.STL10(
        "../datasets", split="test", download=download, transform=transforms.ToTensor()
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=2 * batch_size,
        num_workers=10,
        drop_last=False,
        shuffle=shuffle,
    )
    return train_loader, test_loader


def get_stl10_canny_dual_data_loaders(download, shuffle=False, batch_size=256):
    train_dataset = DualDataset(
        CannyDataset(root=data_root, split="train", transform=transforms.ToTensor()),
        STL10(root=data_root, split="train", transform=transforms.ToTensor()),
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=False,
        shuffle=shuffle,
    )

    test_dataset = DualDataset(
        CannyDataset(root=data_root, split="test", transform=transforms.ToTensor()),
        STL10(root=data_root, split="test", transform=transforms.ToTensor()),
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=2 * batch_size,
        num_workers=10,
        drop_last=False,
        shuffle=shuffle,
    )

    return train_loader, test_loader


def get_stl10_dexined_dual_data_loaders(download, shuffle=False, batch_size=256):
    train_dataset = DualDataset(
        DexiNedTrainDataset(
            csv_file="../Edge_images/Dexi/train/labels.csv",
            root_dir="../Edge_images/Dexi/train",
            transform=transforms.ToTensor(),
        ),
        STL10(root=data_root, split="train", transform=transforms.ToTensor()),
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=False,
        shuffle=shuffle,
    )

    test_dataset = DualDataset(
        DexiNedTestDataset(
            csv_file="../Edge_images/Dexi/test/labels.csv",
            root_dir="../Edge_images/Dexi/test",
            transform=transforms.ToTensor(),
        ),
        STL10(root=data_root, split="test", transform=transforms.ToTensor()),
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=2 * batch_size,
        num_workers=10,
        drop_last=False,
        shuffle=shuffle,
    )

    return train_loader, test_loader

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()

        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k)
        return res

In [ ]:
arch = "resnet18"
print("Backbone arch: ", arch)

exps = [
    "baseline_pretrained",
    "baseline_random",
    "baseline_canny_random",
    "baseline_dexined_random",
    "canny_single_pretrained",
    "canny_single_random",
    "dexined_single_pretrained",
    "dexined_single_random",
    "canny_dual_pretrained",
    "canny_dual_random",
    "dexined_dual_pretrained",
    "dexined_dual_random",
]

for exp in exps:
    print("[INFO] Experiment: ", exp)

    if exp.startswith("baseline"):
        if arch == "resnet18":
            eval_model = torchvision.models.resnet18(pretrained=False).to(device)
        elif arch == "resnet50":
            eval_model = torchvision.models.resnet50(pretrained=False).to(device)

        checkpoint_file_path = os.path.join(
            checkpoint_root,
            checkpoints[exp],
            "checkpoint_" + checkpoints[exp] + ".pth.tar",
        )
        checkpoint = torch.load(checkpoint_file_path, map_location=device)

        state_dict = checkpoint["state_dict"]
        for k in list(state_dict.keys()):
            if k.startswith("backbone."):
                if k.startswith("backbone") and not k.startswith("backbone.fc"):
                    # remove prefix
                    state_dict[k[len("backbone.") :]] = state_dict[k]
            del state_dict[k]

        log = eval_model.load_state_dict(state_dict, strict=False)
        assert log.missing_keys == ["fc.weight", "fc.bias"]

        eval_model.fc = nn.Identity()

    elif "single" in exp:
        init_type = exp.split("_")[-1]
        edge_checkpoint_key, non_edge_checkpoint_key = exp, "baseline_{}".format(
            init_type
        )
        edge_checkpoint_file_path = os.path.join(
            checkpoint_root,
            checkpoints[exp],
            "checkpoint_" + checkpoints[exp] + ".pth.tar",
        )
        non_edge_checkpoint_file_path = os.path.join(
            checkpoint_root,
            checkpoints[non_edge_checkpoint_key],
            "checkpoint_" + checkpoints[non_edge_checkpoint_key] + ".pth.tar",
        )
        eval_model = MorphCLRSingleEval(
            base_model=arch,
            edge_checkpoint_file_path=edge_checkpoint_file_path,
            non_edge_checkpoint_file_path=non_edge_checkpoint_file_path,
            device=device,
            return_embedding=True,
        )
    else:
        checkpoint_file_path = os.path.join(
            checkpoint_root,
            checkpoints[exp],
            "checkpoint_" + checkpoints[exp] + ".pth.tar",
        )
        eval_model = MorphCLRDualEval(
            base_model=arch,
            checkpoint_file_path=checkpoint_file_path,
            device=device,
            return_embedding=True,
        )

    if exp.startswith("baseline"):
        get_data_loader_fn = get_stl10_data_loaders
    elif "canny" in exp:
        get_data_loader_fn = get_stl10_canny_dual_data_loaders
    else:
        get_data_loader_fn = get_stl10_dexined_dual_data_loaders

    train_loader, test_loader = get_data_loader_fn(download=True)

    # freeze all layers
    for name, param in eval_model.named_parameters():
        param.requires_grad = False

    epochs = 1
    n_class = 10
    n_points_per_class = 800
    sample_size = 80
    embeddings_by_cls = {c: [] for c in range(n_class)}

    eval_model.eval()

    for epoch in range(epochs):
        for counter, batch_data in enumerate(tqdm(test_loader)):
            if exp.startswith("baseline"):
                x_batch, y_batch = batch_data
                x_batch = x_batch.to(device)
            else:
                x_edge_batch, x_non_edge_batch, y_batch = batch_data
                x_edge_batch = x_edge_batch.to(device)
                if len(x_edge_batch.shape) == 3:
                    x_edge_batch = x_edge_batch.unsqueeze(1)
                # If the image is of grayscale, repeat the dimension to create 3 channels
                if x_edge_batch.shape[1] == 1:
                    x_edge_batch = x_edge_batch.repeat(1, 3, 1, 1)
                x_non_edge_batch = x_non_edge_batch.to(device)
                x_batch = torch.stack([x_edge_batch, x_non_edge_batch], dim=0)

            y_batch = y_batch.flatten().to(device)

            if not issubclass(type(eval_model), MorphCLRBase):
                embeddings = eval_model(x_batch)
            else:
                _, embeddings = eval_model(x_batch)

            for i, c in enumerate(y_batch.detach().cpu().numpy()):
                embeddings_by_cls[c].append(embeddings[i].detach().cpu().numpy())

    for c in embeddings_by_cls.keys():
        embeddings_by_cls[c] = np.vstack(embeddings_by_cls[c])
        idxs = np.random.choice(range(n_points_per_class), sample_size, replace=False)
        embeddings_by_cls[c] = embeddings_by_cls[c][idxs]

    # Prepare data for t-SNE plots
    tsne_embeddings = np.concatenate([embeddings_by_cls[c] for c in range(n_class)])
    tsne_targets = np.array(
        list(itertools.chain(*[[c] * sample_size for c in range(n_class)]))
    )

    # 2D t-SNE plots
    print("[INFO] Fitting 2D t-SNE plot.")
    tsne = TSNE(2, verbose=1)
    tsne_proj = tsne.fit_transform(tsne_embeddings)
    fig, ax = plt.subplots(figsize=(8, 8))
    cmap = cm.get_cmap("tab20")
    num_categories = 10
    for lab in range(num_categories):
        indices = tsne_targets == lab
        ax.scatter(
            tsne_proj[indices, 0],
            tsne_proj[indices, 1],
            c=np.array(cmap(lab)).reshape(1, 4),
            label=lab + 1,
            alpha=0.5,
        )
    ax.legend(fontsize="large", markerscale=2)
    plt.savefig("./2d_tsne/{}.png".format(exp), bbox_inches="tight")